In [8]:
# --- Evaluation on x-pixel shifted test set ---
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Load the test dataset
transform = transforms.Compose([
    transforms.Resize((227, 227)), # Resize for AlexNet
    transforms.ToTensor(),
    # Standard CIFAR-10 normalization
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616))
])

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


# Load the trained model (from scratch)
model_scratch = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
model_scratch.classifier[6] = torch.nn.Linear(in_features=4096, out_features=10)
model_scratch = model_scratch.to(device)

def shift_images(images, shift_x=10, shift_y=10):
    """
    Shift a batch of images by shift_x and shift_y pixels.
    Pads empty areas with zeros.
    """
    B, C, H, W = images.shape
    shifted = torch.zeros_like(images)
    # shift pixels down and right
    shifted[:, :, shift_y:, shift_x:] = images[:, :, :H-shift_y, :W-shift_x]
    return shifted

shift_pixels = 10
model_scratch.eval()
correct_shifted = 0
total_shifted = 0

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        shifted_images = shift_images(images, shift_x=shift_pixels, shift_y=shift_pixels)
        outputs = model_scratch(shifted_images)
        _, predicted = torch.max(outputs, 1)
        total_shifted += labels.size(0)
        correct_shifted += (predicted == labels).sum().item()

shifted_accuracy = 100 * correct_shifted / total_shifted
print(f"\nAccuracy of AlexNet Scratch on {shift_pixels}-pixel shifted test images: {shifted_accuracy:.2f}%")



Accuracy of AlexNet Scratch on 10-pixel shifted test images: 11.98%
